# Dependencies

In [1]:
import dendropy
import numpy as np
import pandas as pd
from collections import Counter
import itertools as it

# 1. Check tree for unique labels

In [2]:
# Read tree file
tree = dendropy.Tree.get(path="tree.nwk", schema="newick")

In [3]:
####### TESTING
from dendropy import Node, Taxon

n_copies = 10

# Get ancestral node
anc = tree.nodes()[0]

for i in range(n_copies+1):
    # Add a new taxon
    taxon = Taxon("ZACH")

    # Add taxon to namespace
    tree.taxon_namespace.add_taxon(taxon)

    # Add taxon node to tree
    new_node = Node(taxon=taxon)
    anc.add_child(new_node)

In [4]:
def remove_redundant_taxa(tree):
    leafs = tree.leaf_nodes()
    leaf_labels = tree.taxon_namespace.labels()

    # Check that all leafs are unique
    total = 0
    for label, count in Counter(leaf_labels).items():
        total += count-1
        for i in range(count-1):
            node = tree.find_node_with_taxon_label(label)
            tree.prune_nodes([node])

    if total > 0: print("{} taxa were redundant, so I removed them.".format(total))
    return tree

tree = remove_redundant_taxa(tree)

10 taxa were redundant, so I removed them.


# 2. Give internal nodes labels

In [5]:
def check_nodes_are_labelled(tree):
    internal = tree.internal_nodes()
    for i, node in enumerate(internal):
        node.label = "anc{}".format(i)     
    for node in tree.leaf_nodes():
        node.label = node.taxon.label
    return tree

tree = check_nodes_are_labelled(tree)

# 3. Calculate pairwise distance between all nodes

In [6]:
def pairwise_distance_matrix(tree):
    dist = tree.node_distance_matrix()
    nodes = tree.nodes()
    node_labels = [n.label for n in nodes]

    matrix = pd.DataFrame(
        index=node_labels, 
        columns=node_labels)

    for i in range(len(nodes)):
        for j in range(len(nodes)):
            node1 = nodes[i]
            node2 = nodes[j]
            matrix[node1.label][node2.label] = dist(node1, node2)
    return matrix

matrix = pairwise_distance_matrix(tree)

# 4. Pairwise combinations on sides of branch

In [7]:
# Bundles nodes an each side of the branch
branch = tree.edges()[3]
dij = branch.length
head = branch.head_node

# Get all nodes on leftmost side of branch
side_one = []
for n in head.preorder_iter(lambda x: x.is_leaf()):
    if n == branch.tail_node:
        break
    elif n.is_leaf():
        side_one.append(n)

# Get other nodes
side_two = list(set(tree.leaf_nodes()).difference(side_one))

# 5. Sort pairwise combination

In [8]:
pairs_strad = [(n1, n2) for n1 in side_one for n2 in side_two]
pairs_side1 = list(it.combinations(side_one, 2))
pairs_side2 = list(it.combinations(side_two, 2)) 
pairs_same_side = pairs_side1 + pairs_side2

n_straddle = len(pairs_strad)
n_same_side = len(pairs_same_side)

rbca = np.empty(n_straddle+n_same_side, dtype=float)

# 6. Calculate distances for straddle pairs

In [9]:
dist = tree.node_distance_matrix()

# Distance between two leaf nodes
dbc = np.array([matrix[n1.label][n2.label] for n1, n2 in pairs_strad])

# Distance from leaf node to candidate branch
dbi = np.array([dist(head,n1) for n1, n2 in pairs_strad])

# Find optimal position for node on branch
rho = np.sum((dbc-2*dbi)*dbc**-2)/(2*dij*np.sum(dbc**-2))
rho = min(max(0, rho), 1)

# Calculate distance from leaf to branch
dab = dbi + dij*rho
rbca[:n_straddle] = np.abs(2*dab/dbc-1)

# 7. Calculate distances for same side pairs

In [10]:
dbc_same = np.array([matrix[n1.label][n2.label] for n1, n2 in pairs_same_side])
dab_same = np.array([dist(head,n1) for n1, n2 in pairs_same_side])
rbca[n_straddle:] = np.abs(2*dab_same/dbc_same-1)

# 8. Calculate deviations

In [18]:
x = pd.Series(index=tree.edges())

In [19]:
x[tree.edges()[0]]

nan

In [15]:
edge.label

# 8. Calculate the root ambiguity index 